In [4]:
import pandas as pd
from scipy.stats import shapiro, ttest_1samp

In [2]:
data_path = '/hdd/data/adp_data/student_lifestyle_dataset.csv'

df = pd.read_csv(data_path, index_col = 'Student_ID')
df = df.sample(30, random_state = 526)
df.head()

,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,Stress_Level,Gender,Grades
Student_ID,,,,,,,,
1892,8.0,2.5,7.7,0.5,5.3,Moderate,Female,7.80
1983,6.3,2.4,8.1,0.9,6.3,Moderate,Female,7.68
26,9.7,1.3,7.6,3.8,1.6,High,Female,8.22
153,8.5,3.7,8.5,0.5,2.8,High,Male,9.12
1766,8.3,1.5,8.6,1.9,3.7,High,Female,7.15


In [3]:
stat, p = shapiro(df['Sleep_Hours_Per_Day'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=0.939, p=0.084


In [7]:
t_stat, p = ttest_1samp(df['Sleep_Hours_Per_Day'], 9)
print('t-statistic=%.3f, p=%.3f' % (t_stat, p))

t-statistic=-4.096, p=0.000
